In [1]:
import pandas as pd
import re

In [39]:
path_report_oi = './reports/oferta_individual.xlsx'
path_portafolio = './reports/portafolio.xlsx'
path_past_portafolio = './reports/past_portafolio.xlsx'

In [40]:
#Lista de campos que se usarán del reporte estadístico de Hércules.
fields_report = [
    'Evento',
    'Documento',
    'Tipo Documento',
    'Sedes',
    'Área',
    'Ciclos',
    'Categoría Deportiva',
    'Nivel',
    'Género',
    'Categoría Precio',
    'Categoría Afiliación',
    'creada_por',
    'Estado Cotización',
    'Canal de Pago',
    'Canal de Cotización',
    'Forma de Pago',
    'Franquicias',
    'Precio Base',
    'Costo',
    'Tarifa Plena',
    'Subsidio a la Demanda',
    'Identificador del Cajero',
    'Identificador Máquina',
    'Fecha Registro',
    'Documento Titular',
    'Fecha Cotización',
    'Id Material',
    'Fecha Inicio del Servicio',
    'Fecha Fin del Servicio',
    'Id Servicio',
    'Edad',
    'Fecha de Nacimiento',
    'Valor',
    'Motivo de Cambio',
]

In [41]:
# Importando la data desde el reporte.
dataset=pd.read_excel(path_report_oi, usecols=fields_report)

In [43]:
# Se eliminan todos los registros donde no haya un valor en la sede o en la fecha de reserva.
dash_dataset = dataset.dropna(subset=['Sedes', 'Fecha Inicio del Servicio'])
print(f"Se han eliminado {dataset['Evento'].count()-dash_dataset['Evento'].count()} filas que no tienen registros en las sedes o en la fecha de reserva.")


# Inserción de columna 'Tipo de Oferta' al final del dataset, asignádole un valor ''.
dash_dataset.insert(loc=len(dash_dataset.columns), column='Tipo de Oferta', value='')

# Eliminación del prefijo '-F' en los código SAP (campo 'Id Material') de la oferta FOSFEC.
dash_dataset['Id Material'].replace(r'(\d{8})-F', r'\g<1>', inplace=True, regex=True)
dash_dataset['Id Material'] = pd.to_numeric(dash_dataset['Id Material'])

Se han eliminado 112 filas que no tienen registros para las sedes o en la fecha de reserva.


/tmp/ipykernel_75773/1385572998.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_75773/1385572998.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
# Patrones empleados en el campo 'Motivo de cambio' usando RegEx y con ello asignar las ofertas por Excedentes del 55% y cobro a empresas por una venta individual.
pat_exc55 = r'exc\w*\s*\d*[%]*[^excel]'
pat_empresarial = r'empr\w*\s*|conv\w*\s*|nit\s*\d*|contr|se factura'

In [45]:
# Asignación tipo de oferta 'Pago con novedad'.
dash_dataset.loc[dash_dataset['Canal de Pago'].isna(), 'Tipo de Oferta']='Pago con Novedad'

# Asignación tipo de oferta 'FOSFEC'.
dash_dataset.loc[dash_dataset['Evento']=='fosfec', 'Tipo de Oferta'] = 'FOSFEC'

# Asignación tipo de oferta 'Pago empresarial'.
dash_dataset.loc[dash_dataset['Motivo de Cambio'].notna() & dash_dataset['Motivo de Cambio'].str.contains(pat_empresarial, case=False, regex=True), 'Tipo de Oferta'] = 'Pago Empresarial'

# Asignación tipo de oferta 'Excedentes del 55%'.
dash_dataset.loc[(dash_dataset['Ciclos'].str.contains(pat_exc55, case=False, regex=True)) | dash_dataset['Motivo de Cambio'].str.contains(pat_exc55, case=False, regex=True), 'Tipo de Oferta'] = 'Excedentes del 55%'

# Asignación tipo de oferta 'Venta Directa'.
dash_dataset.loc[(dash_dataset['Canal de Pago'].notna()) & (dash_dataset['Motivo de Cambio'].isna()), 'Tipo de Oferta'] = 'Venta Directa'

In [47]:
# Lista de campos del portafolio que se usarán para cruzar con el reporte estadístico.
fields_portafolio = [
    'UNIDAD DE NEGOCIO',
    'LINEA',
    'CODIGO SAP',
    'MATERIAL',
    'NUM. PARTICIPANTES MAX',
]

In [48]:
# última versión del portafolio.
portafolio = pd.read_excel(path_portafolio, usecols=fields_portafolio)

# Archivo que contiene materiales pasados que no están en la última actualización del portafolio.
past_portafolio = pd.read_excel(path_past_portafolio, usecols=fields_portafolio)

full_portafolio=pd.concat([portafolio, past_portafolio])


# Selecciona del portafolio sólo las filas donde la unidad de negocio coincida con las indicacadas en la lista 'UNIDADES DE NEGOCIO', pues son las unidades que ofertan servicios en Hércules.
UNIDADES_NEGOCIO = ['EDUCACIÓN', 'CULTURA', 'ESPARCIMIENTO', 'DESARROLLO SOCIAL']
portafolio_hercules = full_portafolio[
     full_portafolio['UNIDAD DE NEGOCIO'].isin(UNIDADES_NEGOCIO)
]

# Elimina las fila donde se encuentre el repetido el código SAP.
portafolio_hercules.drop_duplicates('CODIGO SAP', inplace=True, keep='first')

/tmp/ipykernel_75773/865904377.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [110]:
portafolio_hercules.to_csv('full.csv', index=False, index_label=False)

In [49]:
# Se ejecuta un left-join trayendo los campos de portafolio cruzados a través del Código SAP.
final_dataset = pd.merge(
    dash_dataset,
    portafolio_hercules[
        [
            'UNIDAD DE NEGOCIO',
            'LINEA',
            'CODIGO SAP',
            'NUM. PARTICIPANTES MAX',
        ]
    ],
    left_on='Id Material',
    right_on='CODIGO SAP',
    how='left',
    # validate='1:m'
    
)

In [123]:
# final_dataset.to_csv('dash_dataset.csv', index = False, index_label=False)

In [2]:
# df=pd.read_csv('./assets/data/dash_dataset.csv', header=0)

In [190]:


final=final_dataset.groupby(by=["Mes Registro"], as_index=True).count().pivot(columns=["Tipo de oferta"])
final
# final=final_dataset[["Tipo de Oferta", "Mes Registro"]]
# final.groupby("Mes Registro").count()

KeyError: "None of ['Tipo de oferta'] are in the columns"

In [27]:
from datetime import date
import plotly.express as px
# figure=px.line(data, x=data.date, y="GOOG")
e

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472
